# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un conjunto de documentos de texto.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [1]:
import time
# Variable con las rutas de los documentos en la carpeta "corpus"
corpus = [
    '../data/01_corpus_turismo.txt',
    '../data/01_corpus_turismo_500.txt',
    # Puedes agregar más documentos aquí
]

def buscar_query_en_corpus(query, corpus):
    documentos_con_query = []

    for doc in corpus:
        with open(doc, 'r', encoding='utf-8') as archivo:
            contenido = archivo.read()
            palabras = contenido.split()  # Separa el contenido en palabras

            for word in palabras:
                if word == query:
                    documentos_con_query.append(doc)
                    break  # Ya encontramos la query en este documento

    return documentos_con_query

def buscar_query_en_corpus_con_tiempo(query, corpus):
    inicio = time.time()
    resultados = buscar_query_en_corpus(query, corpus)
    fin = time.time()
    tiempo = fin - inicio
    return resultados, tiempo

# Ejemplo de uso
if __name__ == "__main__":
    query = input("Ingresa la palabra (query) que deseas buscar: ")
    resultado = buscar_query_en_corpus(query, corpus)

    if resultado:
        print("\nLa palabra aparece en los siguientes documentos:")
        for doc in resultado:
            print(f"- {doc}")
    else:
        print("\nLa palabra no se encontró en ningún documento.")



La palabra aparece en los siguientes documentos:
- ../data/01_corpus_turismo.txt
- ../data/01_corpus_turismo_500.txt


## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

In [3]:
import string
import time

def construir_indice_invertido(corpus):
    indice_invertido = {}

    for doc in corpus:
        with open(doc, 'r', encoding='utf-8') as archivo:
            for num_linea, linea in enumerate(archivo, start=1):
                palabras = linea.split()

                for palabra in palabras:
                    # Limpiar signos de puntuación
                    palabra = palabra.strip(string.punctuation)

                    if palabra:
                        if palabra not in indice_invertido:
                            indice_invertido[palabra] = {}

                        if doc not in indice_invertido[palabra]:
                            indice_invertido[palabra][doc] = []

                        indice_invertido[palabra][doc].append(num_linea)

    return indice_invertido

def buscar_query_en_indice(query, indice_invertido):
    return indice_invertido.get(query, {})

def buscar_indice_inverso_con_tiempo(query, indice_inverso):
    inicio = time.time()
    resultados = indice_inverso.get(query, {})
    fin = time.time()
    tiempo = fin - inicio
    return resultados, tiempo

# Ejemplo de uso
if __name__ == "__main__":
    # Define tu corpus aquí
    corpus = ['../data/01_corpus_turismo.txt', '../data/01_corpus_turismo_500.txt']

    indice_invertido = construir_indice_invertido(corpus)

    # Mostrar el índice invertido completo
    print("\nÍndice Invertido:")
    for palabra, documentos in indice_invertido.items():
        print(f"{palabra}: {documentos}")

    # Buscar en el índice
    query = input("\nIngresa la palabra (query) que deseas buscar: ")
    resultado = buscar_query_en_indice(query, indice_invertido)

    if resultado:
        print("\nLa palabra aparece en los siguientes documentos y líneas:")
        for doc, lineas in resultado.items():
            print(f"- {doc}: líneas {lineas}")
    else:
        print("\nLa palabra no se encontró en ningún documento.")



Índice Invertido:
Ecuador: {'../data/01_corpus_turismo.txt': [1], '../data/01_corpus_turismo_500.txt': [5, 22, 30, 37, 41, 45, 76, 109, 110, 122, 127, 130, 156, 163, 168, 200, 225, 231, 246, 254, 258, 280, 307, 319, 324, 333, 352, 371, 379, 383, 386, 442, 459, 486, 491, 492]}
es: {'../data/01_corpus_turismo.txt': [1, 6, 9], '../data/01_corpus_turismo_500.txt': [1, 5, 7, 8, 10, 14, 22, 29, 30, 32, 33, 34, 37, 41, 45, 53, 57, 60, 65, 72, 76, 78, 101, 106, 109, 110, 111, 115, 118, 119, 122, 127, 130, 138, 149, 154, 156, 159, 161, 163, 168, 171, 177, 180, 187, 190, 191, 193, 200, 204, 207, 212, 216, 222, 223, 225, 231, 246, 248, 250, 254, 256, 258, 264, 271, 275, 280, 289, 293, 294, 297, 307, 309, 319, 324, 331, 333, 334, 335, 340, 341, 342, 345, 349, 351, 352, 358, 363, 365, 367, 371, 373, 379, 380, 383, 386, 388, 410, 412, 420, 437, 442, 447, 448, 454, 456, 459, 460, 462, 463, 465, 468, 477, 482, 486, 488, 491, 492, 496]}
un: {'../data/01_corpus_turismo.txt': [1, 3, 9], '../data/01_corp

## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño: 16 documentos (turismo en Ecuador).
      -  Corpus grande: 500 documentos (versión ampliada).
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

### Ejemplo de palabras para buscar
- quito
- montañita
- feriado
- playas
- aventura
- galápagos

In [ ]:
# Listado de palabras a buscar
palabras_a_buscar = ['quito', 'montañita', 'feriado', 'playas', 'aventura', 'galápagos']

# Corpus con los documentos
corpus = [
    'data/01_corpus_turismo.txt',  # Documento pequeño
    'data/01_corpus_turismo_500.txt'  # Documento grande
]


# Medir el tiempo de búsqueda para cada palabra y método
for palabra in palabras_a_buscar:
    for doc in corpus:
        # Búsqueda Secuencial
        resultados_sec, tiempo_sec = buscar_query_en_corpus_con_tiempo(palabra, [doc])

        # Búsqueda con Índice Inverso
        resultados_ind, tiempo_ind = buscar_indice_inverso_con_tiempo(palabra, indice_invertido)

        # Imprimir los resultados
        print(f"\nBúsqueda de la palabra '{palabra}' en el documento '{doc}':")
        print(f"- Tiempo de búsqueda secuencial: {tiempo_sec:.6f} segundos")
        print(f"- Tiempo de búsqueda con índice inverso: {tiempo_ind:.6f} segundos")
        print(f"  Resultados encontrados con búsqueda secuencial: {resultados_sec}")
        print(f"  Resultados encontrados con índice inverso: {resultados_ind}")




Búsqueda de la palabra 'quito' en el documento 'data/01_corpus_turismo.txt':
- Tiempo de búsqueda secuencial: 0.006800 segundos
- Tiempo de búsqueda con índice inverso: 0.000000000000000000000000000000000000000000000000000000000000000000 segundos
  Resultados encontrados con búsqueda secuencial: []
  Resultados encontrados con índice inverso: {}

Búsqueda de la palabra 'quito' en el documento 'data/01_corpus_turismo_500.txt':
- Tiempo de búsqueda secuencial: 0.001064 segundos
- Tiempo de búsqueda con índice inverso: 0.000000000000000000000000000000000000000000000000000000000000000000 segundos
  Resultados encontrados con búsqueda secuencial: []
  Resultados encontrados con índice inverso: {}

Búsqueda de la palabra 'montañita' en el documento 'data/01_corpus_turismo.txt':
- Tiempo de búsqueda secuencial: 0.000000 segundos
- Tiempo de búsqueda con índice inverso: 0.000000000000000000000000000000000000000000000000000000000000000000 segundos
  Resultados encontrados con búsqueda secuenci

## Parte 4:
### Actividad
1. Modifica el índice para que ignore mayúsculas/minúsculas (por ejemplo, "Playa" y "playa" deben considerarse iguales).
2. Permite consultas de múltiples términos (ejemplo: buscar documentos que contengan "playa" y "turismo").
3. Calcula el _speedup_

In [ ]:
import string
import time

def construir_indice_invertido(corpus):
    indice_invertido = {}

    for doc in corpus:
        with open(doc, 'r', encoding='utf-8') as archivo:
            for num_linea, linea in enumerate(archivo, start=1):
                palabras = linea.split()

                for palabra in palabras:
                    # Limpiar signos de puntuación y pasar a minúsculas
                    palabra = palabra.strip(string.punctuation).lower()

                    if palabra:
                        if palabra not in indice_invertido:
                            indice_invertido[palabra] = {}

                        if doc not in indice_invertido[palabra]:
                            indice_invertido[palabra][doc] = []

                        indice_invertido[palabra][doc].append(num_linea)

    return indice_invertido

def buscar_query_en_indice(query, indice_invertido):
    query = query.lower()
    return indice_invertido.get(query, {})

def buscar_queries_multiples(queries, indice_invertido):
    resultados = {}
    for query in queries:
        query = query.lower()
        documentos = indice_invertido.get(query, {})
        for doc, lineas in documentos.items():
            if doc not in resultados:
                resultados[doc] = set()
            resultados[doc].update(lineas)
    # Convertimos los sets en listas y ordenamos las líneas
    for doc in resultados:
        resultados[doc] = sorted(list(resultados[doc]))
    return resultados

def buscar_indice_inverso_con_tiempo(query, indice_invertido):
    inicio = time.time()
    resultados = buscar_query_en_indice(query, indice_invertido)
    fin = time.time()
    tiempo = fin - inicio
    return resultados, tiempo

def buscar_multiples_con_tiempo(queries, indice_invertido):
    inicio = time.time()
    resultados = buscar_queries_multiples(queries, indice_invertido)
    fin = time.time()
    tiempo = fin - inicio
    return resultados, tiempo

# Ejemplo de uso
if __name__ == "__main__":
    corpus = ['data/01_corpus_turismo.txt', 'data/01_corpus_turismo_500.txt']

    indice_invertido = construir_indice_invertido(corpus)

    # Mostrar el índice invertido completo
    print("\nÍndice Invertido:")
    for palabra, documentos in indice_invertido.items():
        print(f"{palabra}: {documentos}")

    # Buscar en el índice
    query = input("\nIngresa la palabra o palabras (separadas por espacio) que deseas buscar: ").strip()

    if ' ' in query:
        queries = query.split()
        resultados, tiempo_multiple = buscar_multiples_con_tiempo(queries, indice_invertido)

        if resultados:
            print("\nLas palabras aparecen en los siguientes documentos y líneas (combinadas):")
            for doc, lineas in resultados.items():
                print(f"- {doc}: líneas {lineas}")
        else:
            print("\nNo se encontraron documentos que contengan esas palabras.")

        print(f"\nTiempo de búsqueda múltiple: {tiempo_multiple:.6f} segundos")

    else:
        resultado, tiempo_simple = buscar_indice_inverso_con_tiempo(query, indice_invertido)

        if resultado:
            print("\nLa palabra aparece en los siguientes documentos y líneas:")
            for doc, lineas in resultado.items():
                print(f"- {doc}: líneas {lineas}")
        else:
            print("\nLa palabra no se encontró en ningún documento.")

        print(f"\nTiempo de búsqueda simple: {tiempo_simple:.6f} segundos")

    # Si quieres calcular el speedup
    if ' ' in query:
        speedup = tiempo_simple / tiempo_multiple if tiempo_multiple > 0 else float('inf')
        print(f"\nSpeedup de la búsqueda simple vs múltiple: {speedup:.2f}x")



Índice Invertido:
ecuador: {'data/01_corpus_turismo.txt': [1], 'data/01_corpus_turismo_500.txt': [5, 22, 30, 37, 41, 45, 76, 109, 110, 122, 127, 130, 156, 163, 168, 200, 225, 231, 246, 254, 258, 280, 307, 319, 324, 333, 352, 371, 379, 383, 386, 442, 459, 486, 491, 492]}
es: {'data/01_corpus_turismo.txt': [1, 6, 9], 'data/01_corpus_turismo_500.txt': [1, 5, 7, 8, 10, 14, 22, 29, 30, 32, 33, 34, 37, 41, 45, 53, 57, 60, 65, 72, 76, 78, 101, 106, 109, 110, 111, 115, 118, 119, 122, 127, 130, 138, 149, 154, 156, 159, 161, 163, 168, 171, 177, 180, 187, 190, 191, 193, 200, 204, 207, 212, 216, 222, 223, 225, 231, 246, 248, 250, 254, 256, 258, 264, 271, 275, 280, 289, 293, 294, 297, 307, 309, 319, 324, 331, 333, 334, 335, 340, 341, 342, 345, 349, 351, 352, 358, 363, 365, 367, 371, 373, 379, 380, 383, 386, 388, 410, 412, 420, 437, 442, 447, 448, 454, 456, 459, 460, 462, 463, 465, 468, 477, 482, 486, 488, 491, 492, 496]}
un: {'data/01_corpus_turismo.txt': [1, 3, 9], 'data/01_corpus_turismo_500.txt